<a href="https://colab.research.google.com/github/Thunga269/Computer-Vision/blob/main/Cats_and_dogs_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import, division, print_function
#nhét thư viện vào này
import os
#tensorflow phiên bản 1.10.0 nhé
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#tải dataset vê này
zip_file = tf.keras.utils.get_file(origin="https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip",
                                   fname="cats_and_dogs_filtered.zip", extract=True)
#get đường dẫn của file mà các bạn đã tải về này
base_dir, _ = os.path.splitext(zip_file)

#cấu hình đường dẫn của thư mục training này
train_dir = os.path.join(base_dir, 'train')
#cấu hình đường dẫn của thư mục validate này
validation_dir = os.path.join(base_dir, 'validation')
print('day la duong dan: ')
print(validation_dir)
# Cấu hình đường dẫn của thư mục chưa ảnh mèo này
train_cats_dir = os.path.join(train_dir, 'cats')
print ('Total training cat images:', len(os.listdir(train_cats_dir)))

# Cấu hình đường dẫn của thư mục chưa ảnh chó này
train_dogs_dir = os.path.join(train_dir, 'dogs')
print ('Total training dog images:', len(os.listdir(train_dogs_dir)))

#Cấu hình đường dẫn của thư mục validate chứa ảnh mèo này
validation_cats_dir = os.path.join(validation_dir, 'cats')
print ('Total validation cat images:', len(os.listdir(validation_cats_dir)))

# Cấu hình đường dẫn của thư mục dùng để validate chứa ảnh chó
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
print ('Total validation dog images:', len(os.listdir(validation_dogs_dir)))

image_size = 160 # tất cả ảnh được resize về 160x160
#số sample được đưa vào xử lý
batch_size = 32
#sinh thêm nhiều dạng dữ liệu nữa sử dụng ImageDataGenerator của tensorflow
# Rescale image
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                #  sử dụng binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

IMG_SHAPE = (image_size, image_size, 3)

# load base model từ  MobileNet đã được huấn luyện trước rồi này
base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

#thêm các layer ở lớp cao nhất
# base_model.summary()
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])
#sigmoid active function thường dùng cho binary classification

model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# model.summary()

epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size
"""
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
"""
history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()




TensorFlow version is  2.8.0
68616192/68606236 [==============================] - 0s 0us/step
day la duong dan: 
/root/.keras/datasets/cats_and_dogs_filtered/validation
Total training cat images: 1000
Total training dog images: 1000
Total validation cat images: 500
Total validation dog images: 500
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
17235968/17225924 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
62/62 [==============================] - 59s 909ms/step - loss: 0.6620 - accuracy: 0.6453 - val_loss: 0.5318 - val_accuracy: 0.7500
Epoch 2/10
62/62 [==============================] - 55s 885ms/step - loss: 0.4500 - accuracy: 0.8034 - val_loss: 0.3721 - val_accuracy: 0.8518
Epoch 3/10
62/62 [==============================] - 55s 882ms/step - loss: 0.3252 - accuracy: 0.8836 - val_loss: 0.2774 - val_accuracy: 0.9052
Epoch 4/10
62/62 [==============================] - 56s 886ms/step - loss: 0.2466 - accuracy: 0.9233 - val_loss: 0.2159 - val_accuracy: 0.9365
Epoch 5/10
62/62 [==============================] - 55s 883ms/step - loss: 0.1977 - accuracy: 0.9405 - val_loss: 0.1776 - val_accuracy: 0.9486
Epoch 6/10
62/62 [==============================] - 55s 880ms/step - loss: 0.1670 - accuracy: 0.9507 - val_loss: 0.1498 - val_accuracy: 0.9556
Epoch 7/10
62/62 [==============================] - 56s 893ms/step - loss: 0.1441 - accuracy: 0.9548 - val_loss: 0.1313 - val_accuracy: 0.9556

KeyError: ignored